In [6]:
from pyomo.environ import *
import pandas as pd

In [7]:
dem = pd.read_excel('C:/Jupyter/TESIS/instancias/instancia1.xlsx', sheet_name='dem')
cap = pd.read_excel('C:/Jupyter/TESIS/instancias/instancia1.xlsx', sheet_name='cap')
preco = pd.read_excel('C:/Jupyter/TESIS/instancias/instancia1.xlsx', sheet_name='precos')

In [8]:
dem_cor = {}
for i in range(dem.shape[0]):
    dem_cor[(dem.iloc[i,0], dem.iloc[i,1], dem.iloc[i,2], dem.iloc[i,3])] = dem.iloc[i,4]

cap_cor = {}
for i in range(cap.shape[0]):
    cap_cor[cap.iloc[i,0]] = cap.iloc[i,1]

preco_cor = {}
for i in range(preco.shape[0]):
    preco_cor[(preco.iloc[i,0], preco.iloc[i,1], preco.iloc[i,2])] = preco.iloc[i,3]

In [9]:
n = 4
dem_cor_1 = {(i,j,c,d): dem_cor[(i,j,c,d)] if (i,j,c,d) in dem_cor else 0  for i in range(1, n+1) for j in range(1, n+1) for c in range(1, 2+1) for d in range(0, 3+1)}
preco_cor_1 = {(i,j,c): preco_cor[(i,j,c)] if (i,j,c) in preco_cor else 0  for i in range(1, n+1) for j in range(1, n+1) for c in range(1, 2+1)}

In [10]:
# instanciar o modelo
model = AbstractModel()

n = 4
# Escalares
model.n = Param(initialize = n)
model.i = Param(initialize = n)
model.j = Param(initialize = n)
model.k = Param(initialize = 2)
model.t = Param(initialize = 3)

# conjuntos
model.I = RangeSet(1, model.i)
model.J = RangeSet(1, model.j)
model.K = RangeSet(1, model.k)
model.T = RangeSet(0, model.t)

#parametros indexados
model.P = Param(model.I, model.J, model.K, initialize = preco_cor_1)
model.d = Param(model.I, model.J, model.K, model.T, initialize = dem_cor_1)
model.Q = Param(model.K, initialize = cap_cor)

# variaveis de desicao
model.A = Var(RangeSet(0, model.i), model.J, model.K, model.T, within = NonNegativeIntegers)
model.X = Var(model.I, model.J, model.K, model.T, within = NonNegativeIntegers)
model.Y = Var(model.I, model.J, model.K, model.T, within = NonNegativeIntegers)
model.ar = Var(RangeSet(0, model.j), model.K, model.T, within = NonNegativeIntegers)

# funcao objetivo
def fo(model):
    return sum( model.P[i,j,k]*model.A[i,j,k,model.t-t] for i in model.I for j in model.J for k in model.K for t in model.T)
model.OBJ = Objective(rule = fo, sense = maximize)


# restricoes
def demanda1(model, j, k, t):
    if j >= 2 and t > 0:
        return model.ar[j,k,model.t-t] ==  model.ar[j-1,k,model.t] + model.X[j-1,j,k,model.t-t] + model.Y[j-1,j,k,model.t-t] - model.A[j-2,j-1,k,model.t-t]
    else:
        return Constraint.Skip
model.ConstrainDemanda1 = Constraint(model.J, model.K, model.T, rule = demanda1)


def demanda2(model, j, k, t):
    if j >= 2 and t > 0:
        return model.ar[j,k,model.t-t] <= model.Q[k] - sum( model.A[i,jj,k,model.t-t] for i in model.I for jj in model.J if jj == i+1) 
    else:
        return Constraint.Skip
model.ConstrainDemanda2 = Constraint(model.J, model.K, model.T, rule = demanda2)


def capaci(model, k):
    return sum( model.A[i,j,k,model.t-t] for i in model.I for j in model.J for t in model.T if i<j and t>0) >= model.Q[k]
model.ConstrainCapaci = Constraint(model.K, rule = capaci)


def arrive(model, j, k, t):
    if j >= 2 and j < model.n and t > 0:
        return model.ar[j,k,model.t-t] >= model.A[j-1,j,k,model.t-t]
    else:
        return Constraint.Skip
model.ConstrainArrive = Constraint(model.J, model.K, model.T, rule = arrive)


def arrivelast(model, k, t):
    if t > 0:
        return model.ar[model.n,k,model.t-t] == model.A[model.n-1,model.n,k,model.t-t]
    else:
        return Constraint.Skip
model.ConstrainArriveLast = Constraint(model.K, model.T, rule = arrivelast)


def demanda(model, j, k, t):
    if j >= 2 and j < model.n and t > 0:
        return model.ar[j,k,model.t-t] >= sum( model.d[i,j,k,model.t-t] for i in model.I if i<j) 
    else:
        return Constraint.Skip
model.ConstrainDemanda = Constraint(model.J, model.K, model.T, rule = demanda)


def asignacoe(model, j, k, t):
    if j >= 2 and t > 0:
        return model.X[j-1,j,k,model.t-t] <= model.Y[j-1,j,k,model.t-t]
    else:
        return Constraint.Skip
model.ConstrainAsignacoe = Constraint(model.J, model.K, model.T, rule = asignacoe)


def arriveAsig(model, j, k, t):
    if j >= 2 and t > 0:
        return model.A[j-1,j,k,model.t-t] <= model.X[j-1,j,k,model.t-t]
    else:
        return Constraint.Skip
model.ConstrainArriveAsig = Constraint(model.J, model.K, model.T, rule = arriveAsig)


def asignacoe0(model, k, t):
    if t > 0:
        return model.A[0,1,k,model.t-t] == 0
    else:
        return Constraint.Skip
model.ConstrainAsignacoe0 = Constraint(model.K, model.T, rule = asignacoe0)


def arriveSum(model):
    return sum( model.A[i,j,k,model.t] for i in model.I for j in model.J for k in model.K if j >= 2 and j == i+1 ) == 0
model.ConstrainArriveSum = Constraint(rule = arriveSum)


def arrive0(model, k, t):
    if t > 0:
        return model.ar[0,k,model.t-t] == 0
    else:
        return Constraint.Skip
model.ConstrainArrive0 = Constraint(model.K, model.T, rule = arrive0)



instance = model.create_instance()

opt = SolverFactory('cplex')

results = opt.solve(instance, tee=True)

# instance.pprint()
print(results)


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\Wilmer\AppData\Local\Temp\tmpwswekdxf.cplex.log' open.
CPLEX> Problem 'C:\Users\Wilmer\AppData\Local\Temp\tmpqqd1ssnq.pyomo.lp' read.
Read time = 0.00 sec. (0.01 ticks)
CPLEX> Problem name         : C:\Users\Wilmer\AppData\Local\Temp\tmpqqd1ssnq.pyomo.lp
Objective sense      : Maximize
Variables            :     120  [General Integer: 120]
Objective nonzeros   :      48
Linear constraints   :     117  [Less: 66,  Greater: 14,  Equal: 37]
  Nonzeros           :     336
  RHS nonzeros       :      28

Variables            : Min LB: 0.000000         Max UB: all infinite   
Objective nonz